In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
path = r'C:\Users\User\Movie Industry Analysis'

### Source dataframe

In [2]:
df = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'movies.csv'), index_col = False)

In [3]:
df.columns 

Index(['name', 'rating', 'genre', 'year', 'released', 'score', 'votes',
       'director', 'writer', 'star', 'country', 'budget', 'gross', 'company',
       'runtime'],
      dtype='object')

In [4]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7668 entries, 0 to 7667
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      7668 non-null   object 
 1   rating    7591 non-null   object 
 2   genre     7668 non-null   object 
 3   year      7668 non-null   int64  
 4   released  7666 non-null   object 
 5   score     7665 non-null   float64
 6   votes     7665 non-null   float64
 7   director  7668 non-null   object 
 8   writer    7665 non-null   object 
 9   star      7667 non-null   object 
 10  country   7665 non-null   object 
 11  budget    5497 non-null   float64
 12  gross     7479 non-null   float64
 13  company   7651 non-null   object 
 14  runtime   7664 non-null   float64
dtypes: float64(5), int64(1), object(9)
memory usage: 898.7+ KB


###  The column "released" contents mixed both date and country. I don't need that and will delete the column

In [5]:
df.drop('released', axis=1, inplace=True)

In [6]:
df.shape

(7668, 14)

## 1. Cleaning dataset

### Duplicates

In [7]:
df_dups = df[df.duplicated()]
df_dups

,name,rating,genre,year,score,votes,director,writer,star,country,budget,gross,company,runtime


### No duplicates

### Test for mixed data

In [8]:
for col in df.columns.tolist():
  md = (df[[col]].applymap(type) != df[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df[md]) > 0:
    print (col)


rating
writer
star
country
company


In [9]:
# What is the problem in the "country" column?
df.country.unique()

array(['United Kingdom', 'United States', 'South Africa', 'West Germany',
       'Canada', 'Australia', 'Libya', 'Italy', 'Soviet Union', 'France',
       'Hong Kong', 'South Korea', 'Sweden', 'Spain', 'Mexico',
       'Switzerland', 'Netherlands', 'New Zealand', 'Philippines',
       'Israel', 'Japan', 'Hungary', 'Brazil', 'Yugoslavia', 'Argentina',
       'Taiwan', 'Ireland', 'Denmark', 'Germany', 'Iran', 'China',
       'Belgium', 'Austria', 'Portugal', 'Vietnam',
       'Republic of Macedonia', 'Russia',
       'Federal Republic of Yugoslavia', 'Greece', 'Czech Republic',
       'Norway', 'India', 'Jamaica', 'Aruba', 'Thailand', 'Colombia',
       'Romania', nan, 'United Arab Emirates', 'Indonesia', 'Turkey',
       'Serbia', 'Poland', 'Kenya', 'Finland', 'Iceland', 'Chile',
       'Panama', 'Malta', 'Lebanon'], dtype=object)

### There is 

### Mixed data in the columns "writer", "star", "country", and "company" will not create problems

In [10]:
df.rating.unique()

array(['R', 'PG', 'G', nan, 'Not Rated', 'NC-17', 'Approved', 'TV-PG',
       'PG-13', 'Unrated', 'X', 'TV-MA', 'TV-14'], dtype=object)

### I will change values 'Not Rated' in a column "rating" to 'Unrated'

In [11]:
df.loc[df['rating'] == 'Not Rated', 'rating'] = 'Unrated'

In [12]:
df.rating.unique()

array(['R', 'PG', 'G', nan, 'Unrated', 'NC-17', 'Approved', 'TV-PG',
       'PG-13', 'X', 'TV-MA', 'TV-14'], dtype=object)

### Test for missing data

In [13]:
df.isnull().sum() 

name           0
rating        77
genre          0
year           0
score          3
votes          3
director       0
writer         3
star           1
country        3
budget      2171
gross        189
company       17
runtime        4
dtype: int64

In [14]:
# Clean 'score'
df_clean = df[df['score'].isnull() == False] 

In [15]:
df_clean.shape

(7665, 14)

In [16]:
# Clean 'votes'
df_clean = df_clean[df_clean['votes'].isnull() == False]
df_clean.shape

(7665, 14)

In [17]:
# Clean 'runtime'
df_clean = df_clean[df_clean['runtime'].isnull() == False]
df_clean.shape

(7661, 14)

In [18]:
# Median value for 'gross' 
df_clean['gross'] = df_clean['gross'].fillna(df_clean['gross'].median())

In [19]:
# Clean 'company'
df_clean = df_clean[df_clean['company'].isnull() == False]
df_clean.shape

(7648, 14)

In [20]:
df_clean.isnull().sum() 

name           0
rating        71
genre          0
year           0
score          0
votes          0
director       0
writer         3
star           1
country        1
budget      2160
gross          0
company        0
runtime        0
dtype: int64

###  2169 values of the 'budget' are missing. I need to enter their values. 

In [21]:
df_clean.describe()

,year,score,votes,budget,gross,runtime
count,7648.000000,7648.000000,7.648000e+03,5.488000e+03,7.648000e+03,7648.000000
mean,2000.402197,6.392338,8.827188e+04,3.563924e+07,7.723003e+07,107.279681
std,11.138422,0.968137,1.634614e+05,4.147028e+07,1.641001e+08,18.582572
min,1980.000000,1.900000,7.000000e+00,3.000000e+03,3.090000e+02,55.000000
25%,1991.000000,5.800000,9.200000e+03,1.000000e+07,4.713798e+06,95.000000
50%,2000.000000,6.500000,3.300000e+04,2.100000e+07,2.018496e+07,104.000000
75%,2010.000000,7.100000,9.325000e+04,4.500000e+07,7.312031e+07,116.000000
max,2020.000000,9.300000,2.400000e+06,3.560000e+08,2.847246e+09,366.000000


###  We see that this variable has a very big range: from 3,000 to 356,000,000.
###  We cannot use the mean values. I will use the median values instead. 

In [22]:
df_clean['budget'].median()

21000000.0

In [23]:
df_clean['budget'] = df_clean['budget'].fillna(df_clean['budget'].median())

In [24]:
df_clean.isnull().sum() 

name         0
rating      71
genre        0
year         0
score        0
votes        0
director     0
writer       3
star         1
country      1
budget       0
gross        0
company      0
runtime      0
dtype: int64

### Right now we don't have any missing data. 

In [25]:
# Export the df_clean dataset
# df_clean.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'df_clean.csv'), index=False)    

## 2. Data profile

In [26]:
# Descriptive Statistics
df_clean.describe() 
#df_clean1=df_clean

,year,score,votes,budget,gross,runtime
count,7648.000000,7648.000000,7.648000e+03,7.648000e+03,7.648000e+03,7648.000000
mean,2000.402197,6.392338,8.827188e+04,3.150473e+07,7.723003e+07,107.279681
std,11.138422,0.968137,1.634614e+05,3.574136e+07,1.641001e+08,18.582572
min,1980.000000,1.900000,7.000000e+00,3.000000e+03,3.090000e+02,55.000000
25%,1991.000000,5.800000,9.200000e+03,1.400000e+07,4.713798e+06,95.000000
50%,2000.000000,6.500000,3.300000e+04,2.100000e+07,2.018496e+07,104.000000
75%,2010.000000,7.100000,9.325000e+04,3.200000e+07,7.312031e+07,116.000000
max,2020.000000,9.300000,2.400000e+06,3.560000e+08,2.847246e+09,366.000000


### Considering the min/max values of numerical variables there are no values outside of reasonable range
###    

In [27]:
# data types in a table
df_clean1.dtypes

NameError: name 'df_clean1' is not defined

In [ ]:
# Change type of 'year' from int64 to text
df_clean1['year'] = df_clean1['year'].astype('str') 

In [ ]:
df_clean1.dtypes

In [ ]:
# Export the df_clean1 dataset
df_clean1.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'df_clean2.csv'), index=False)    

In [ ]:
df_clean2 = df_clean1

In [ ]:
# Descriptive Statistics after change of type
df_clean2.describe()

In [ ]:
df_clean2.info() 

### Generate correlation matrix

In [ ]:
# Create a new dataframe from df_clean2 contains only numerical values
df_numeric = df_clean2.drop(columns=['name', 'rating', 'genre', 'year', 'director', 'writer', 'star', 'country', 'company'])

In [ ]:
df_numeric.corr()

In [ ]:
# Export the df_numeric dataset
df_numeric.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'df_numeric_.csv'), index=False)    

In [ ]:
# Numbers of unique values
df_clean2.nunique()

In [ ]:
# Modes for columns
modes = df_clean2.mode()
modes.head(10)

In [ ]:
# Medians for columns
print(df_clean2.median(numeric_only=True))